In [1]:
import numpy as np
import pandas as pd
from RecGroupSys import VotingMethods
from tqdm import tqdm


In [2]:
df_users = pd.read_csv('../data/df_users_train2.csv')

In [3]:
df_groups = pd.read_pickle('df_groups_svdpp_random_popular_xgb')
df_groups.head()

,members,members2,preds_svdpp,preds_random,preds_popular,preds_xgb
0,"['Snawk', 'Opal82']","[Snawk, Opal82]",161936 314040 182028 224517 0 8...,223040 25021 314040 241477 194655 319...,161936 224517 233078 187645 182028 115...,BGGId 91 25021 ...
1,"['David546', 'taragalinas']","[David546, taragalinas]",266507 139976 175914 61487 0 8....,113997 26997 61487 126163 234 824...,161936 224517 233078 187645 182028 115...,BGGId 277085 1320...
2,"['Vadorojo', 'Travellingmatti', 'DanKill']","[Vadorojo, Travellingmatti, DanKill]",246900 139976 237182 324856 2...,300010 295488 24508 171131 17329 205...,161936 224517 233078 187645 182028 115...,BGGId 272453 ...
3,"['HeavyAdge', 'Jawaswag']","[HeavyAdge, Jawaswag]",233571 61487 227460 12333 0 10.0 ...,193738 327 257501 207572 206480 301...,161936 224517 233078 187645 182028 115...,BGGId 244331 256952...
4,"['Qelha14', 'MarcusK']","[Qelha14, MarcusK]",120677 224517 2651 12333 0 9.5...,183562 93260 131111 21050 33643 120...,161936 224517 233078 187645 182028 115...,BGGId 302723 37141 ...


In [64]:

RELEVANCE_THRESHOLD = 5


def precision_at_k(recommendations_df, user_history_df, k=10, threshold=RELEVANCE_THRESHOLD):
    recommended_ids = list(recommendations_df.index)
    if len(recommendations_df) > k:
        recommended_ids = recommended_ids[:k]
    relevant_items = user_history_df[user_history_df['Rating'] >= threshold]['BGGId']
    hits = pd.Series(recommended_ids).isin(relevant_items).sum()
    return hits / k


def ndcg_at_k(recommendations_df, user_history_df, k=10):
    recommended_ids = list(recommendations_df.index)
    if len(recommendations_df) > k:
        recommended_ids = recommended_ids[:k]

    actual_k = len(recommended_ids)

    if actual_k == 0:
        return 0.0

    temp_recs_df = pd.DataFrame({'BGGId': recommended_ids})
    merged_df = pd.merge(temp_recs_df, user_history_df, on='BGGId', how='left')

    relevance = merged_df['Rating'].fillna(1).values

    dcg = np.sum(relevance / np.log2(np.arange(2, actual_k + 2)))

    ideal_relevance = np.sort(relevance)[::-1]
    idcg = np.sum(ideal_relevance / np.log2(np.arange(2, actual_k + 2)))

    if idcg == 0:
        return 0.0

    return dcg / idcg


results = []

for grp_id in tqdm(range(df_groups.shape[0])):
    users = df_groups.loc[grp_id, 'members2']
    groups_results = []

    for prd in ['preds_svdpp', 'preds_random', 'preds_popular', 'preds_xgb']:

        aux = df_groups.loc[grp_id, prd].fillna(1)
        for aggmethod in [VotingMethods.average, VotingMethods.multiplicative,
                          VotingMethods.borda_count, VotingMethods.copeland_score, VotingMethods.approval_voting,
                          VotingMethods.least_misery, VotingMethods.most_pleasure, VotingMethods.average_without_misery,
                          VotingMethods.fairness, VotingMethods.most_respected_person]:
            _pr = []
            _nd = []

            rdf = aggmethod(aux)
            if aggmethod.__name__ == 'fairness':
                lx = np.linspace(10, 1, len(rdf))
                rdf = pd.Series(lx, index=rdf)
            rdf = rdf.sort_values(ascending=False)

            for idx, usr_name in enumerate(users):
                usr_hist = df_users[(df_users.Username == usr_name)&(df_users.isTest)]
                usr_hist = usr_hist.loc[usr_hist.groupby('BGGId')['Rating'].idxmax()]

                _pr.append(precision_at_k(rdf, usr_hist))
                _nd.append(ndcg_at_k(rdf, usr_hist))

            groups_results.append({prd: {aggmethod.__name__: {'precision': np.mean(_pr), 'ndcg': np.mean(_nd)}}})
    results.append(groups_results)

print('Done')


100%|██████████| 247/247 [04:45<00:00,  1.16s/it]

Done


In [65]:
flat_data = []
for run_data in results:
    for item in run_data:
        model_name = list(item.keys())[0]
        method_data = item[model_name]
        method_name = list(method_data.keys())[0]
        metrics = method_data[method_name]
        flat_data.append({
            "model": model_name,
            "method": method_name,
            "precision": metrics["precision"],
            "ndcg": metrics["ndcg"]
        })

# Crear el DataFrame
df = pd.DataFrame(flat_data)
df

,model,method,precision,ndcg
0,preds_svdpp,average,0.3,0.984312
1,preds_svdpp,multiplicative,0.3,0.866594
2,preds_svdpp,borda_count,0.3,0.922970
3,preds_svdpp,copeland_score,0.3,0.866594
4,preds_svdpp,approval_voting,0.3,0.866594
...,...,...,...,...
9875,preds_xgb,least_misery,0.0,1.000000
9876,preds_xgb,most_pleasure,0.0,1.000000
9877,preds_xgb,average_without_misery,0.0,0.000000
9878,preds_xgb,fairness,0.0,1.000000


In [86]:
# Agrupar y calcular promedio y desviación estándar
# agg_results = df.groupby(['model', 'method']).agg(['mean', 'std'])
agg_results = df.groupby(['model', 'method']).agg([
    ('mean', lambda x: x[x != 0].mean()),
    ('std', lambda x: x[x != 0].std())
])

def create_metric_table(agg_df, metric_name):
    """Formatea la tabla para una métrica específica (mean ± std)."""
    # Seleccionar los datos de la métrica
    metric_df = agg_df[metric_name]

    # Formatear la celda como 'promedio ± std'
    # .fillna(0) se usa por si hay un caso con una sola medición (std = NaN)
    formatted_series = (
        metric_df['mean'].map('{:.3f}'.format) +
        ' ± ' +
        metric_df['std'].fillna(0).map('{:.3f}'.format)
    )

    # Reorganizar la tabla a la forma final: modelos en filas, métodos en columnas
    return formatted_series.unstack('method')

# --- Generar y mostrar las tablas ---

# Tabla para Precision
precision_table = create_metric_table(agg_results, 'precision')
print("--- Tabla de Resultados: Precision (Promedio ± Std) ---")
display(precision_table)

# Tabla para NDCG
ndcg_table = create_metric_table(agg_results, 'ndcg')
print("\n--- Tabla de Resultados: NDCG (Promedio ± Std) ---")
display(ndcg_table)

--- Tabla de Resultados: Precision (Promedio ± Std) ---


method,approval_voting,average,average_without_misery,borda_count,copeland_score,fairness,least_misery,most_pleasure,most_respected_person,multiplicative
model,,,,,,,,,,
preds_popular,0.402 ± 0.169,0.402 ± 0.169,0.246 ± 0.114,0.402 ± 0.169,0.402 ± 0.169,0.402 ± 0.169,0.402 ± 0.169,0.402 ± 0.169,0.402 ± 0.169,0.402 ± 0.169
preds_random,0.160 ± 0.077,0.158 ± 0.083,0.090 ± 0.022,0.158 ± 0.083,0.165 ± 0.098,0.158 ± 0.083,0.156 ± 0.080,0.159 ± 0.083,0.167 ± 0.096,0.156 ± 0.080
preds_svdpp,0.156 ± 0.083,0.156 ± 0.083,0.118 ± 0.040,0.156 ± 0.083,0.156 ± 0.083,0.156 ± 0.083,0.156 ± 0.083,0.156 ± 0.083,0.156 ± 0.083,0.156 ± 0.083
preds_xgb,0.074 ± 0.044,0.067 ± 0.029,nan ± 0.000,0.058 ± 0.028,0.072 ± 0.032,0.061 ± 0.030,0.073 ± 0.043,0.070 ± 0.027,0.073 ± 0.031,0.067 ± 0.029



--- Tabla de Resultados: NDCG (Promedio ± Std) ---


method,approval_voting,average,average_without_misery,borda_count,copeland_score,fairness,least_misery,most_pleasure,most_respected_person,multiplicative
model,,,,,,,,,,
preds_popular,0.741 ± 0.136,0.741 ± 0.136,0.817 ± 0.161,0.741 ± 0.136,0.741 ± 0.136,0.741 ± 0.136,0.741 ± 0.136,0.741 ± 0.136,0.741 ± 0.136,0.741 ± 0.136
preds_random,0.796 ± 0.170,0.799 ± 0.169,1.000 ± 0.000,0.797 ± 0.170,0.788 ± 0.167,0.796 ± 0.168,0.811 ± 0.173,0.797 ± 0.169,0.789 ± 0.168,0.811 ± 0.173
preds_svdpp,0.861 ± 0.158,0.849 ± 0.168,0.999 ± 0.003,0.858 ± 0.159,0.858 ± 0.159,0.852 ± 0.161,0.858 ± 0.159,0.843 ± 0.169,0.856 ± 0.159,0.858 ± 0.159
preds_xgb,0.980 ± 0.059,0.993 ± 0.035,1.000 ± 0.000,0.975 ± 0.064,0.983 ± 0.053,0.976 ± 0.063,0.979 ± 0.058,0.994 ± 0.034,0.983 ± 0.053,0.993 ± 0.035


In [87]:
precision_table.T


model,preds_popular,preds_random,preds_svdpp,preds_xgb
method,,,,
approval_voting,0.402 ± 0.169,0.160 ± 0.077,0.156 ± 0.083,0.074 ± 0.044
average,0.402 ± 0.169,0.158 ± 0.083,0.156 ± 0.083,0.067 ± 0.029
average_without_misery,0.246 ± 0.114,0.090 ± 0.022,0.118 ± 0.040,nan ± 0.000
borda_count,0.402 ± 0.169,0.158 ± 0.083,0.156 ± 0.083,0.058 ± 0.028
copeland_score,0.402 ± 0.169,0.165 ± 0.098,0.156 ± 0.083,0.072 ± 0.032
fairness,0.402 ± 0.169,0.158 ± 0.083,0.156 ± 0.083,0.061 ± 0.030
least_misery,0.402 ± 0.169,0.156 ± 0.080,0.156 ± 0.083,0.073 ± 0.043
most_pleasure,0.402 ± 0.169,0.159 ± 0.083,0.156 ± 0.083,0.070 ± 0.027
most_respected_person,0.402 ± 0.169,0.167 ± 0.096,0.156 ± 0.083,0.073 ± 0.031


In [88]:
ndcg_table.T

model,preds_popular,preds_random,preds_svdpp,preds_xgb
method,,,,
approval_voting,0.741 ± 0.136,0.796 ± 0.170,0.861 ± 0.158,0.980 ± 0.059
average,0.741 ± 0.136,0.799 ± 0.169,0.849 ± 0.168,0.993 ± 0.035
average_without_misery,0.817 ± 0.161,1.000 ± 0.000,0.999 ± 0.003,1.000 ± 0.000
borda_count,0.741 ± 0.136,0.797 ± 0.170,0.858 ± 0.159,0.975 ± 0.064
copeland_score,0.741 ± 0.136,0.788 ± 0.167,0.858 ± 0.159,0.983 ± 0.053
fairness,0.741 ± 0.136,0.796 ± 0.168,0.852 ± 0.161,0.976 ± 0.063
least_misery,0.741 ± 0.136,0.811 ± 0.173,0.858 ± 0.159,0.979 ± 0.058
most_pleasure,0.741 ± 0.136,0.797 ± 0.169,0.843 ± 0.169,0.994 ± 0.034
most_respected_person,0.741 ± 0.136,0.789 ± 0.168,0.856 ± 0.159,0.983 ± 0.053


In [99]:
grp = df_groups.loc[1,'members2']
grp

['David546', 'taragalinas']